<a href="https://colab.research.google.com/github/villamroland/EA/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# LSTM for energy consumption problem with time step regression framing
import numpy
import matplotlib.pyplot as plt
import copy
import pandas as pd
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [0]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [0]:
numpy.random.seed(69)

In [4]:
dataframe = pd.read_csv('NI_hourly.csv', sep=',')

print('train set: {0}'.format(dataframe.shape))
dataframe.head(10)

train set: (58450, 2)


,Datetime,NI_MW
0,2004-12-31 01:00:00,9810.0
1,2004-12-31 02:00:00,9001.0
2,2004-12-31 03:00:00,8509.0
3,2004-12-31 04:00:00,8278.0
4,2004-12-31 05:00:00,8089.0
5,2004-12-31 06:00:00,8208.0
6,2004-12-31 07:00:00,8431.0
7,2004-12-31 08:00:00,8891.0
8,2004-12-31 09:00:00,9098.0
9,2004-12-31 10:00:00,9237.0


In [0]:
dataframe['Datetime']
dataframe['Year'] = dataframe['Datetime']
dataframe['Month'] = dataframe['Datetime']
dataframe['Day'] = dataframe['Datetime']
dataframe['Hour'] = dataframe['Datetime']

def getDays(dates):
    temp = []

    for i in range(0,len(dates)):
        temp.append((int)(''.join(dates[i].split(sep='-')[2]).split(sep=' ')[0]))

    return temp


def getMonths(dates):
    temp = []

    for i in range(0,len(dates)):
        temp.append((int)(dates[i].split(sep='-')[1]))

    return temp

def getYears(dates):
    temp = []

    for i in range(0,len(dates)):
        temp.append((int)(dates[i].split(sep='-')[0]))

    return temp

def getHours(dates):
    temp = []

    for i in range(0,len(dates)):
        temp.append((int)(''.join(''.join(dates[i].split(sep='-')[2]).split(sep=' ')[1])[0:2]))

    return temp

dates = copy.copy(dataframe["Datetime"]) #converting date time

dates = dates.values

dataframe.Day = getDays(dates)
dataframe.Month=getMonths(dates)
dataframe.Year=getYears(dates)
dataframe.Hour = getHours(dates)
# for row in dataframe.rows:
#   row['Year'] = row['Datetime']#[0:4] 
#   row['Month'] = row['Datetime']#[5:7]
#   row['Day'] = row['Datetime']#[8:10]
del dataframe['Datetime']


In [6]:
dataframe.tail()

,NI_MW,Year,Month,Day,Hour
58445,13239.0,2010,1,1,20
58446,13156.0,2010,1,1,21
58447,12978.0,2010,1,1,22
58448,12743.0,2010,1,1,23
58449,12223.0,2010,1,2,0


In [0]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataframe = scaler.fit_transform(dataframe)

In [0]:
train_size = int(len(dataframe) * 0.67)
test_size = len(dataframe) - train_size
train, test = dataframe[0:train_size,:], dataframe[train_size:len(dataframe),:]

In [0]:
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [0]:
trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [15]:
model = Sequential()
model.add(LSTM(4, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1)

Epoch 1/100
 8538/39157 [=====>........................] - ETA: 3:42 - loss: 0.0050

KeyboardInterrupt: ignored